In [8]:
import pandas as pd
# Especifica la ruta del archivo CSV que deseas leer
file_path = 'Data/Datos_csv/games.csv'

# Lee el archivo CSV y crea un DataFrame
games = pd.read_csv(file_path, parse_dates=['release_date'])

# Ahora puedes trabajar con el DataFrame 'df'
print(games)

              publisher                                             genres  \
0             Kotoshiro  ['Action', 'Casual', 'Indie', 'Simulation', 'S...   
1      Making Fun, Inc.       ['Free to Play', 'Indie', 'RPG', 'Strategy']   
2          Poolians.com  ['Casual', 'Free to Play', 'Indie', 'Simulatio...   
3                  彼岸领域                  ['Action', 'Adventure', 'Casual']   
4                   NaN                                                NaN   
...                 ...                                                ...   
32128   Ghost_RUS Games      ['Casual', 'Indie', 'Simulation', 'Strategy']   
32129            Sacada                    ['Casual', 'Indie', 'Strategy']   
32130      Laush Studio                  ['Indie', 'Racing', 'Simulation']   
32131          SIXNAILS                                ['Casual', 'Indie']   
32132               NaN                                                NaN   

                       app_name                     title  \
0 

In [9]:
# Especifica la ruta del archivo CSV que deseas leer
file_path = 'Data/Datos_csv/reviews_nlp.csv'

# Lee el archivo CSV y crea un DataFrame
reviews = pd.read_csv(file_path, parse_dates=['posted'])

# Ahora puedes trabajar con el DataFrame 'reviews'
print(reviews)

                 user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
2      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
3                js41637               http://steamcommunity.com/id/js41637   
4                js41637               http://steamcommunity.com/id/js41637   
...                  ...                                                ...   
59300  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
59301  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
59302        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
59303        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
59304        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                                  funny     posted 

In [10]:
# Especifica la ruta del archivo CSV que deseas leer
file_path = 'Data/Datos_csv/df_train_filtrado.csv'

# Lee el archivo CSV y crea un DataFrame
df_train_filtrado = pd.read_csv(file_path)

# Ahora puedes trabajar con el DataFrame 'reviews'
print(df_train_filtrado)

                 user_id  item_id  recommend
0      76561197970982479     1250       True
1      76561197970982479    22200       True
2      76561197970982479    43110       True
3                js41637   251610       True
4                js41637   227300       True
...                  ...      ...        ...
36485  76561198312638244       70       True
36486  76561198312638244   362890       True
36487        LydiaMorley   273110       True
36488        LydiaMorley      730       True
36489        LydiaMorley      440       True

[36490 rows x 3 columns]


In [17]:
pip install scikit-surprise

In [18]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise import accuracy
from surprise import KNNBasic, KNNWithMeans, KNNBaseline
from surprise import SVD

In [12]:
# Configura el lector y la escala de calificación (ajusta esto según tus datos)
reader = Reader(rating_scale=(0, 1))

# Carga los datos
data = Dataset.load_from_df(df_train_filtrado[['user_id', 'item_id', 'recommend']], reader)

# Divide los datos en conjuntos de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)

# Crea el modelo KNN (K-Nearest Neighbors) para encontrar usuarios similares
sim_options = {'name': 'pearson', 'user_based': True}
algo = KNNBasic(sim_options=sim_options)

# Entrena el modelo en el conjunto de entrenamiento
algo.fit(trainset)

# Supongamos que tenemos un usuario de interés con user_id = 'tu_usuario'
tu_usuario = 'evcentric'

# Encuentra los usuarios más similares al usuario de interés
similar_users = algo.get_neighbors(algo.trainset.to_inner_uid(tu_usuario), k=10)

# Filtra las reseñas de los usuarios similares
reviews_similares = reviews[reviews['user_id'].isin([algo.trainset.to_raw_uid(uid) for uid in similar_users])]

# Filtra las reseñas de juegos que han sido recomendados (recommend=True)
juegos_recomendados = reviews_similares[reviews_similares['recommend']]

# Obtén la lista de juegos recomendados por usuarios similares
juegos_recomendados_lista = juegos_recomendados['item_id'].unique()

# Imprime la lista de juegos recomendados
print("Juegos recomendados por usuarios similares:")
print(juegos_recomendados_lista)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Juegos recomendados por usuarios similares:
[312210 218620 265550 252490 303210   4000  24240 204300 236390 418910
   8170   8850 203160   8870  17300   7670 270210 206210 230410 284460
 259080 339800 323370 257030 212070    570     70  15320 314160  72850
 200710    730     80 210170    440]


In [13]:
# Realiza predicciones en el conjunto de prueba
predictions = algo.test(testset)

# Calcula la RMSE para evaluar el rendimiento del modelo
rmse = accuracy.rmse(predictions)

# Imprime la RMSE
print(f'RMSE: {rmse}')

RMSE: 0.3296
RMSE: 0.32956156242933293


In [14]:
# Inicializa una lista vacía para almacenar los juegos encontrados
juegos_encontrados = []

# Itera a través de la lista de juegos recomendados
for juego_id in juegos_recomendados_lista:
    # Busca el juego en el DataFrame games
    juego = games[games['item_id'] == juego_id]
    
    # Si se encuentra el juego, agrégalo a la lista de juegos encontrados
    if not juego.empty:
        juegos_encontrados.append(juego[['item_id', 'app_name']])  # Agrega solo las columnas "item_id" y "app_name" del juego encontrado

    # Si ya hemos encontrado 5 juegos, detén el bucle
    if len(juegos_encontrados) == 5:
        break

# Convierte la lista de juegos encontrados en un DataFrame
juegos_encontrados_df = pd.concat(juegos_encontrados, ignore_index=True)

# Imprime el DataFrame de juegos encontrados con las columnas "item_id" y "app_name"
print(juegos_encontrados_df[['item_id', 'app_name']])
juegos_encontrados_df

   item_id                          app_name
0   312210                     The Fifth Day
1   218620                          PAYDAY 2
2   265550  Dead Rising 3 Apocalypse Edition
3   252490                              Rust
4   303210              The Beginner's Guide


,item_id,app_name
0,312210,The Fifth Day
1,218620,PAYDAY 2
2,265550,Dead Rising 3 Apocalypse Edition
3,252490,Rust
4,303210,The Beginner's Guide


In [21]:
def similar_user(tu_usuario: str):

    # Encuentra los usuarios más similares al usuario de interés
    similar_users = algo.get_neighbors(algo.trainset.to_inner_uid(tu_usuario), k=10)

    # Filtra las reseñas de los usuarios similares
    reviews_similares = reviews[reviews['user_id'].isin([algo.trainset.to_raw_uid(uid) for uid in similar_users])]

    # Filtra las reseñas de juegos que han sido recomendados (recommend=True)
    juegos_recomendados = reviews_similares[reviews_similares['recommend']]

    # Obtén la lista de juegos recomendados por usuarios similares
    juegos_recomendados_lista = juegos_recomendados['item_id'].unique()

    # Imprime la lista de juegos recomendados
    # print("Juegos recomendados por usuarios similares:")
    # print(juegos_recomendados_lista)


    # Inicializa una lista vacía para almacenar los juegos encontrados
    juegos_encontrados = []

    # Itera a través de la lista de juegos recomendados
    for juego_id in juegos_recomendados_lista:
        # Busca el juego en el DataFrame games
        juego = games[games['item_id'] == juego_id]
        
        # Si se encuentra el juego, agrégalo a la lista de juegos encontrados
        if not juego.empty:
            juegos_encontrados.append(juego[['item_id', 'app_name']])  # Agrega solo las columnas "item_id" y "app_name" del juego encontrado

        # Si ya hemos encontrado 5 juegos, detén el bucle
        if len(juegos_encontrados) == 5:
            break

    # Convierte la lista de juegos encontrados en un DataFrame
    juegos_encontrados_df = pd.concat(juegos_encontrados, ignore_index=True)

    # Imprime el DataFrame de juegos encontrados con las columnas "item_id" y "app_name"
    #return print(juegos_encontrados_df[['item_id', 'app_name']])
    return juegos_encontrados

In [23]:
similar_user('evcentric')

[       item_id       app_name
 28667   312210  The Fifth Day,
        item_id  app_name
 30068   218620  PAYDAY 2,
        item_id                          app_name
 28444   265550  Dead Rising 3 Apocalypse Edition,
        item_id app_name
 29691   252490     Rust,
        item_id              app_name
 25439   303210  The Beginner's Guide]

In [20]:
juegos_encontrados

[       item_id       app_name
 28667   312210  The Fifth Day,
        item_id  app_name
 30068   218620  PAYDAY 2,
        item_id                          app_name
 28444   265550  Dead Rising 3 Apocalypse Edition,
        item_id app_name
 29691   252490     Rust,
        item_id              app_name
 25439   303210  The Beginner's Guide]